In [1]:
# Swapnil Saha Shawon (2022533042)
# Tamanna Rahman (2021450642)
# Syeda Mashiat Tabassum (2031356642)

In [2]:
#include libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

#load data and checking contents
data = pd.read_csv("car.data.csv")
print(data)

#check for null values
print("\nChecking NULL values:\n",data.isnull().sum())

     buying  maint  doors persons lug_boot safety  class
0     vhigh  vhigh      2       2    small    low  unacc
1     vhigh  vhigh      2       2    small    med  unacc
2     vhigh  vhigh      2       2    small   high  unacc
3     vhigh  vhigh      2       2      med    low  unacc
4     vhigh  vhigh      2       2      med    med  unacc
...     ...    ...    ...     ...      ...    ...    ...
1723    low    low  5more    more      med    med   good
1724    low    low  5more    more      med   high  vgood
1725    low    low  5more    more      big    low  unacc
1726    low    low  5more    more      big    med   good
1727    low    low  5more    more      big   high  vgood

[1728 rows x 7 columns]

Checking NULL values:
 buying      0
maint       0
doors       0
persons     0
lug_boot    0
safety      0
class       0
dtype: int64


In [3]:
#preprocessing data
categorical_cols = data.select_dtypes(include=['object']).columns.tolist()
print("Categorical Columns:", categorical_cols)
print("\n")

#label encoding features
from sklearn.preprocessing import LabelEncoder

labelEncoder = LabelEncoder()

for col in categorical_cols:
  data[col] = labelEncoder.fit_transform(data[col])

print(data)

Categorical Columns: ['buying', 'maint', 'doors', 'persons', 'lug_boot', 'safety', 'class']


      buying  maint  doors  persons  lug_boot  safety  class
0          3      3      0        0         2       1      2
1          3      3      0        0         2       2      2
2          3      3      0        0         2       0      2
3          3      3      0        0         1       1      2
4          3      3      0        0         1       2      2
...      ...    ...    ...      ...       ...     ...    ...
1723       1      1      3        2         1       2      1
1724       1      1      3        2         1       0      3
1725       1      1      3        2         0       1      2
1726       1      1      3        2         0       2      1
1727       1      1      3        2         0       0      3

[1728 rows x 7 columns]


In [4]:
#selecting target variable and features
X = data.drop('class', axis = 1)
y = data['class']

#splitting data for training, validating and testing
X_train = X.iloc[:1209]
X_validation = X.iloc[1209:1468]
X_test = X.iloc[1468:]
print(X_validation)
y_train = y.iloc[:1209]
y_validation = y.iloc[1209:1468]
y_test = y.iloc[1468:]

      buying  maint  doors  persons  lug_boot  safety
1209       2      1      0        2         1       1
1210       2      1      0        2         1       2
1211       2      1      0        2         1       0
1212       2      1      0        2         0       1
1213       2      1      0        2         0       2
...      ...    ...    ...      ...       ...     ...
1463       1      0      2        0         1       0
1464       1      0      2        0         0       1
1465       1      0      2        0         0       2
1466       1      0      2        0         0       0
1467       1      0      2        1         2       1

[259 rows x 6 columns]


In [8]:
import random
import numpy as np
from sklearn import datasets
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Define the fitness function
def fitness_function(params):
    C, gamma = params
    model = SVC(C=C, gamma=gamma)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_validation)
    accuracy = accuracy_score(y_validation, y_pred)
    return accuracy

# Create the initial population
def create_population(size, param_ranges):
    population = []
    for _ in range(size):
        individual = [random.uniform(*param_ranges[param]) for param in param_ranges]
        population.append(individual)
    return population

# Evaluate the population
def evaluate_population(population):
    fitness_scores = [fitness_function(individual) for individual in population]
    return fitness_scores

# Select parents for crossover
def select_parents(population, fitness_scores, num_parents):
    selected = random.choices(population, weights=fitness_scores, k=num_parents)
    return selected

# Crossover between parents to create offspring
def crossover(parents, offspring_size):
    offspring = []
    for _ in range(offspring_size):
        parent1 = random.choice(parents)
        parent2 = random.choice(parents)
        child = [(p1 + p2) / 2 for p1, p2 in zip(parent1, parent2)]
        offspring.append(child)
    return offspring

# Mutate the offspring
def mutate(offspring, mutation_rate, param_ranges):
    for individual in offspring:
        if random.random() < mutation_rate:
            param_idx = random.randint(0, len(individual) - 1)
            individual[param_idx] = random.uniform(*param_ranges[list(param_ranges.keys())[param_idx]])
    return offspring

# Genetic algorithm
def genetic_algorithm(pop_size, num_generations, mutation_rate, param_ranges):
    population = create_population(pop_size, param_ranges)
    for generation in range(num_generations):
        fitness_scores = evaluate_population(population)
        parents = select_parents(population, fitness_scores, pop_size // 2)
        offspring = crossover(parents, pop_size - len(parents))
        offspring = mutate(offspring, mutation_rate, param_ranges)
        population = parents + offspring
        best_individual = max(population, key=fitness_function)
        best_fitness = fitness_function(best_individual)
        print(f'Generation {generation}: Best Fitness = {best_fitness}, Best Individual = {best_individual}')
    return best_individual, best_fitness

# Parameters
population_size = 20
num_generations = 50
mutation_rate = 0.1
param_ranges = {
    'C': (0.1, 100),
    'gamma': (0.001, 1)
}

# Run the genetic algorithm
best_params, best_fitness = genetic_algorithm(population_size, num_generations,
                                              mutation_rate, param_ranges)
print(f'Best hyperparameters: C={best_params[0]}, gamma={best_params[1]}')
print(f'Best validation accuracy: {best_fitness}')

Generation 0: Best Fitness = 0.8996138996138996, Best Individual = [55.07307157685905, 0.19743550926536474]
Generation 1: Best Fitness = 0.8996138996138996, Best Individual = [55.07307157685905, 0.19743550926536474]
Generation 2: Best Fitness = 0.8996138996138996, Best Individual = [55.07307157685905, 0.19743550926536474]
Generation 3: Best Fitness = 0.8996138996138996, Best Individual = [55.07307157685905, 0.19743550926536474]
Generation 4: Best Fitness = 0.8803088803088803, Best Individual = [35.03470560631848, 0.608863533541033]
Generation 5: Best Fitness = 0.8803088803088803, Best Individual = [49.06892844654687, 0.5502450691821166]
Generation 6: Best Fitness = 0.8996138996138996, Best Individual = [39.049745461276586, 0.15768639041768534]
Generation 7: Best Fitness = 0.8803088803088803, Best Individual = [49.06892844654687, 0.5502450691821166]
Generation 8: Best Fitness = 0.8803088803088803, Best Individual = [46.564132700229294, 0.6016735722165751]
Generation 9: Best Fitness = 0.

In [9]:
X_train_full = X.iloc[:1468]
y_train_full = y.iloc[:1468]

# Evaluate the best model on the test set
best_model = SVC(C=best_params[0], gamma=best_params[1])
best_model.fit(X_train_full, y_train_full)  # Train on the full training set
y_test_pred = best_model.predict(X_test)
test_accuracy = accuracy_score(y_test, y_test_pred)
print(f'Test accuracy with best hyperparameters: {test_accuracy}')

Test accuracy with best hyperparameters: 0.8846153846153846
